In [13]:
import matplotlib.pyplot as plt
import numpy as np
from keras.callbacks import EarlyStopping
from keras.layers import Input, Dense
from keras.models import Model, Sequential
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.backend import mean, square

from spektral.datasets import qm9
from spektral.layers import EdgeConditionedConv, GlobalAttentionPool
from spektral.utils import label_to_one_hot

from os import path

In [2]:
A, X, E, y = qm9.load_data(return_type='numpy',
                          nf_keys='atomic_num',
                          ef_keys='type',
                          self_loops=True,
                          amount=10000)
uniq_X = np.unique(X)
X = label_to_one_hot(X, uniq_X)

Loading QM9 dataset.
Reading SDF


100%|██████████| 10000/10000 [00:04<00:00, 2081.63it/s]


In [3]:
tasks = list(y.columns)[1:]
y_list = []
for task in tasks:
    y_list.append(y[[task]].values)
for i in range(len(y_list)):
    y_list[i] = StandardScaler().fit_transform(y_list[i]).reshape(-1, y_list[0].shape[-1])

In [5]:
N = X.shape[-2]
F = X.shape[-1] 
S = E.shape[-1]
n_out = y_list[0].shape[-1]
learning_rate = 1e-3
epochs = 25
batch_size = 64
es_patience = 5

In [6]:
A_train, A_test, \
X_train, X_test, \
E_train, E_test, \
*y_train_test_list = train_test_split(A, X, E, *y_list, test_size = 0.1)

y_train_list = y_train_test_list[::2]
y_test_list = y_train_test_list[1::2]

In [7]:
X_in = Input(shape=(N, F))
A_in = Input(shape=(N, N))
E_in = Input(shape=(N, N, S))

In [8]:
def create_single_task_model(X_in, A_in, E_in):
    gc1 = EdgeConditionedConv(64, activation='relu')([X_in, A_in, E_in])
    gc2 = EdgeConditionedConv(128, activation='relu')([gc1, A_in, E_in])
    pool = GlobalAttentionPool(256)(gc2)
    dense = Dense(256, activation='relu')(pool)
    output = Dense(n_out)(dense)
    return Model(inputs=[X_in, A_in, E_in], outputs=output)

In [14]:
for i in range(len(tasks)):
    print('learning', tasks[i])
    optimizer = Adam(lr=learning_rate)
    model = create_single_task_model(X_in, A_in, E_in)
    model.compile(optimizer=optimizer, loss='mse')
    es_callback = EarlyStopping(monitor='val_loss', patience=es_patience)
    model.fit([X_train, A_train, E_train],
             y_train_list[i],
             batch_size=batch_size,
             validation_split=0.1,
             epochs=epochs,
             callbacks=[es_callback])
    model.save(path.join('single_task_trained_models', tasks[i], '.h5'))

learning A
Train on 8100 samples, validate on 900 samples
Epoch 1/25
6656/8100 [=======================>......] - ETA: 49s - loss: 1.5042

KeyboardInterrupt: 

In [ ]:
def calculate_transfer_coefficient_from_file(task_learned, task_transferred, y_transferred_test):
    model_learned = load_model(path.join('single_task_trained_models', task_learned, '.h5'))
    model_transferred = load_model(path.join('single_task_trained_models', task_transferred, '.h5'))
    layers_learned = model_learned.get_weights()
    layers_transferred = model_transferred.get_weights()
    layer_transferred = learned_layers[:10] + transferred_layers[10:]
    model_transferred.set_weights(layers_transferred)
    eval_results = model_transferred.evaluate([X_test, A_test, E_test],
                                              y_transferred_test,
                                              batch_size=batch_size)
    return eval_results